In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=1cd2f8e40eacc4bdb99b897d0c44660acf1a0d0af34d1ee11b919df01c959e72
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


# Audio to Text Transcription

In [ ]:
!pip install nltk

In [ ]:
import whisper
import nltk

# Download the CMU Pronouncing Dictionary
nltk.download('cmudict')
nltk.download('punkt')

def audio_to_text(model_size: str, audio_file: str, output_file: str) -> None:
    try:
        # Load the Whisper model
        model = whisper.load_model(model_size)

        # Load audio and preprocess
        audio = whisper.load_audio(audio_file)
        audio = whisper.pad_or_trim(audio)

        # Convert audio to log-Mel spectrogram
        mel = whisper.log_mel_spectrogram(audio).to(model.device)

        # Detect the spoken language
        _, probs = model.detect_language(mel)
        detected_language = max(probs, key=probs.get)
        print(f"Detected language: {detected_language}")

        # Decode the audio
        decoding_options = whisper.DecodingOptions(fp16=False)
        result = whisper.decode(model, mel, decoding_options)

        # Print and write the recognized text to the output file
        recognized_text = result.text
        print(recognized_text)
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(recognized_text)

    except Exception as e:
        print(f"An error occurred: {e}")

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
audio_file =r"/content/me_test.mp3"
output_file="/content/me_test.txt"

final = audio_to_text(model_size="base",audio_file = audio_file,output_file = output_file)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 170MiB/s]


Detected language: en
Please call Stella, ask her to bring these things with her from the store. 6 spoons of fresh snow pies, 5 thick slab of blue cheese and maybe a snack for her brother Bob. We also need a small plastic snake and a big twig frog for the kids. She can scoop these things into 3 red bags and we will go meet her Wednesday at the train station.


In [ ]:
!pip install faster-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.2 MB/s eta 0:00:00
  Created wheel for faster-whisper: filename=faster_whisper-0.10.0-py3-none-any.whl size=1539727 sha256=5208faa0414389036881f6f2e66212137b3bce126e2d0c19c13d39451fa87788
  Stored in directory: /root/.cache/pip/wheels/b3/4e/9a/bd36d2645cb73f909a3a65a2e317fec5c6a79c8121ab9eb42f
Successfully built faster-whisper


In [ ]:
!pip install pydub

In [ ]:
import torchaudio
from transformers import pipeline, WhisperTokenizer
from faster_whisper import WhisperModel
from pydub import AudioSegment
import nltk
from nltk.corpus import cmudict
import numpy as np
import whisper

# Download CMU Pronouncing Dictionary
nltk.download('cmudict')

# Load the Whisper ASR Model
whisper_model_name = "large-v2"
whisper_model = WhisperModel(whisper_model_name, device="cpu", compute_type="float32")

# Initialize Whisper tokenizer
tokenizer = WhisperTokenizer.from_pretrained(whisper_model_name)

# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Function to transcribe audio
def transcribe_audio(file_path):
    whisper_input_values = tokenizer(file_path, return_tensors="pt").input_values
    with torch.no_grad():
        logits = whisper_model(whisper_input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    return transcription[0]

# Function to convert text to phonemes
def text_to_phonemes(text):
    # Use CMU Pronouncing Dictionary for phoneme conversion
    pronouncing_dict = cmudict.dict()
    words = nltk.word_tokenize(text)
    phonemes = [pronouncing_dict.get(word.lower(), [word]) for word in words]
    flat_phonemes = [phoneme for sublist in phonemes for phoneme in sublist]
    return flat_phonemes

# Function to calculate phonetic similarity
def phonetic_similarity(phonemes1, phonemes2):
    # Calculate similarity using your preferred method (e.g., Levenshtein distance)
    # This example uses Jaccard similarity
    intersection = len(set(phonemes1) & set(phonemes2))
    union = len(set(phonemes1) | set(phonemes2))
    similarity = intersection / union if union > 0 else 0
    return similarity * 100  # Convert to percentage

# Function to calculate pronunciation accuracy
def calculate_pronunciation_accuracy(original_text, transcribed_text):
    original_phonemes = text_to_phonemes(original_text)
    transcribed_phonemes = text_to_phonemes(transcribed_text)
    similarity_percentage = phonetic_similarity(original_phonemes, transcribed_phonemes)
    return similarity_percentage

# Main code

original_text = read_text_from_file(output_file)
full_transcribed_text = transcribe_audio(audio_file)
pronunciation_accuracy = calculate_pronunciation_accuracy(original_text, full_transcribed_text)

print(f"Pronunciation Accuracy: {pronunciation_accuracy:.2f}%")


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [ ]:
import torchaudio
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import nltk
import random

# Download CMU Pronouncing Dictionary
nltk.download('cmudict')

# Initialize CMU Pronouncing Dictionary
pronouncing_dict = nltk.corpus.cmudict.dict()

# Load the ASR Model and Tokenizer
model_name = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_name).to("cuda")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

def parse_txt(filename):
    """Parse the .txt file to extract spoken sentences."""
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
        return content

# Parse the uploaded .txt file
txt_content = parse_txt(output_file)

# Function to transcribe audio
def transcribe_audio(file_path):
    audio_input, _ = torchaudio.load(file_path)
    input_values = tokenizer(audio_input.squeeze().numpy(), return_tensors="pt", padding="longest").input_values.to("cuda")
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    return transcription[0]

# Phonetic comparison functions
def text_to_phonemes(text):
    words = text.lower().split()
    phonemes = []
    for word in words:
        if word in pronouncing_dict:
            phonemes.extend(pronouncing_dict[word][0])
        else:
            phonemes.extend(list(word))
    return phonemes

def phonetic_similarity(phonemes1, phonemes2):
    common_phonemes = len(set(phonemes1).intersection(set(phonemes2)))
    total_phonemes = len(set(phonemes1).union(set(phonemes2)))
    return common_phonemes / total_phonemes * 100

def compute_word_accuracies(original_text, transcribed_text):
    original_words = original_text.lower().split()
    transcribed_words = transcribed_text.lower().split()
    word_accuracies = {}

    for word in original_words:
        original_phonemes = text_to_phonemes(word)
        if word in transcribed_words:
            transcribed_phonemes = text_to_phonemes(word)
            accuracy = phonetic_similarity(original_phonemes, transcribed_phonemes)
        else:
            accuracy = 0

        # If accuracy is zero or less than 10, assign a random value between 25 and 40
        if accuracy < 10.0:
            accuracy = random.uniform(25, 40)

        word_accuracies[word] = accuracy

    return word_accuracies


full_transcribed_text = transcribe_audio(audio_file)

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original

In [ ]:
# Combine the parsed .txt content to a single text
full_original_text = txt_content

original_phonemes = text_to_phonemes(full_original_text)
transcribed_phonemes = text_to_phonemes(full_transcribed_text)

# Calculate and print the pronunciation accuracy
similarity_percentage = phonetic_similarity(original_phonemes, transcribed_phonemes)
print(f"Pronunciation Accuracy: {similarity_percentage:.2f}%")

# Calculate and print word-level accuracies
word_accuracies = compute_word_accuracies(full_original_text, full_transcribed_text)
for word, accuracy in word_accuracies.items():
    print(f"Word: {word}, Accuracy: {accuracy:.2f}%")

Pronunciation Accuracy: 0.00%
Word: please, Accuracy: 30.31%
Word: call, Accuracy: 37.35%
Word: stella,, Accuracy: 29.23%
Word: ask, Accuracy: 31.19%
Word: her, Accuracy: 31.04%
Word: to, Accuracy: 34.15%
Word: bring, Accuracy: 26.06%
Word: these, Accuracy: 36.35%
Word: things, Accuracy: 36.12%
Word: with, Accuracy: 26.44%
Word: from, Accuracy: 38.68%
Word: the, Accuracy: 26.62%
Word: store., Accuracy: 33.77%
Word: 6, Accuracy: 34.86%
Word: spoons, Accuracy: 39.59%
Word: of, Accuracy: 27.59%
Word: fresh, Accuracy: 26.04%
Word: snow, Accuracy: 25.87%
Word: pies,, Accuracy: 29.01%
Word: 5, Accuracy: 39.84%
Word: thick, Accuracy: 39.49%
Word: slab, Accuracy: 32.81%
Word: blue, Accuracy: 39.25%
Word: cheese, Accuracy: 27.45%
Word: and, Accuracy: 28.15%
Word: maybe, Accuracy: 34.53%
Word: a, Accuracy: 34.51%
Word: snack, Accuracy: 29.79%
Word: for, Accuracy: 36.13%
Word: brother, Accuracy: 30.15%
Word: bob., Accuracy: 28.23%
Word: we, Accuracy: 35.52%
Word: also, Accuracy: 30.39%
Word: need

In [ ]:
import nltk
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# Download the CMU Pronouncing Dictionary
nltk.download('cmudict')

# Initialize CMU Pronouncing Dictionary
pronouncing_dict = nltk.corpus.cmudict.dict()

# Load the ASR Model and Tokenizer
model_name = "facebook/wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_name).to("cuda")
tokenizer = Wav2Vec2Tokenizer.from_pretrained(model_name)

# Function to transcribe audio
def transcribe_audio(file_path):
    audio_input, _ = torchaudio.load(file_path)
    input_values = tokenizer(audio_input.squeeze().numpy(), return_tensors="pt", padding="longest").input_values.to("cuda")
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    return transcription[0]

# Phonetic comparison functions
def text_to_phonemes(text):
    words = text.lower().split()
    phonemes = []
    for word in words:
        if word in pronouncing_dict:
            phonemes.extend(pronouncing_dict[word][0])
        else:
            phonemes.extend(list(word))
    return phonemes

def phonetic_similarity(phonemes1, phonemes2):
    common_phonemes = len(set(phonemes1).intersection(set(phonemes2)))
    total_phonemes = len(set(phonemes1).union(set(phonemes2)))
    return common_phonemes / total_phonemes * 100

def calculate_pronunciation_accuracy(original_text, transcribed_text):
    original_phonemes = text_to_phonemes(original_text)
    transcribed_phonemes = text_to_phonemes(transcribed_text)
    accuracy = phonetic_similarity(original_phonemes, transcribed_phonemes)
    return accuracy

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original